In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import os
import re
from matplotlib import colors

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True) #add flag force_remount = True if needed

Mounted at /content/drive


In [ ]:
!cp -rf "/content/drive/MyDrive/LDA/News.zip" .
!unzip -q News.zip

In [ ]:
df = pd.read_json('News_Category_Dataset_v3.json', orient = 'records', lines = True)
df[['headline', 'category', 'short_description']].head()

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

stopwordset = set(stopwords.words())
# for named entity recognition (NER)
from nltk import ne_chunk

In [ ]:
def clean_text(headline):
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stopwordset]
  cleaned_text=" ".join(tokens)
  return cleaned_text

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df['head_clean']=df['headline'].apply(clean_text)
df['desc_clean']=df['short_description'].apply(clean_text)
df = df.drop(['authors', 'date'], axis = 1)

In [ ]:
df.sample(20)

,link,headline,category,short_description,head_clean,desc_clean
179719,https://www.huffingtonpost.com/entry/my-plea-t...,My Plea to the Supreme Court,PARENTING,"Because I'm a mother, I'm often asked how thes...",My Plea Supreme Court,"Because I 'm mother , I 'm asked school shooti..."
96940,https://www.huffingtonpost.com/entry/hillary-c...,Hillary Clinton's 'Super Volunteers' Are Back ...,POLITICS,,Hillary Clinton 's 'Super Volunteers ' Are Bac...,
165509,https://www.huffingtonpost.com/entry/beard-fac...,The Beard: When Young Men Look Like Old Men,STYLE & BEAUTY,"In the world of facial stubble, the famous fiv...",The Beard : When Young Men Look Like Old Men,"In world facial stubble , famous o'clock shado..."
71599,https://www.huffingtonpost.com/entry/tv-report...,"Female TV Reporters Read The Sexist, Horrible ...",WOMEN,"""You've gained too much weight,"" wrote one vie...","Female TV Reporters Read The Sexist , Horrible...","`` You 've gained weight , '' wrote viewer Chi..."
95222,https://www.huffingtonpost.com/entry/ruth-bade...,Ruth Bader Ginsberg Just Got The Rap Tribute S...,QUEER VOICES,,Ruth Bader Ginsberg Just Got The Rap Tribute S...,
6846,https://www.huffpost.com/entry/brazil-venezuel...,"On The Brazil Border, Venezuela's Political Cr...",WORLD NEWS,"Venezuela’s descent into chaos, which nearly c...","On The Brazil Border , Venezuela 's Political ...","Venezuela ’ descent chaos , culminated anti-go..."
155874,https://www.huffingtonpost.com/entry/huffingto...,Huffington This Week: Change of Heart,PARENTING,"As Rachel writes about her 6 year-old, she rea...",Huffington This Week : Change Heart,"As Rachel writes 6 year-old , realized horror ..."
103190,https://www.huffingtonpost.com/entry/warren-bu...,Warren Buffett's Mobile Home Empire Preys On T...,BUSINESS,,Warren Buffett 's Mobile Home Empire Preys On ...,
66991,https://www.huffingtonpost.comhttp://www.theda...,Dying GOP Senator Apologizes To Muslims For Do...,POLITICS,Bob Bennett spent his last days letting Muslim...,Dying GOP Senator Apologizes To Muslims For Do...,Bob Bennett spent day letting Muslims sorry Is...
163943,https://www.huffingtonpost.com/entry/stunning-...,10 Stunning Terraces And Balconies That Inspir...,HOME & LIVING,These are the perfect places to wind down at t...,10 Stunning Terraces And Balconies That Inspir...,These perfect place wind long day .


In [ ]:
df.shape

(209527, 6)